<a href="https://colab.research.google.com/github/stef4k/train-maintenance-data-mining/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package Importation

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from collections import Counter
import ast
import pickle
from joblib import Parallel, delayed
from google.colab import files
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
from sklearn.utils import parallel_backend
import threading
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from gensim.models import Word2Vec
from scipy.optimize import minimize
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tqdm import tqdm
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

# `Experiment`

This class is a pipeline that automates the process of training and testing using cross-validation and prepares the system for ensemble evaluations. This class was developed as pipeline to do experiments more easily, since we want to test all possible model, sampling strategies and vectorizers available.

One main limitation is the business knowledge we have, since we don not really understand at a very specific level this dataset, like what exactly each incident is, or what does each event in the events sequence represent, we didn't take any apriori assumtios of the data, and we decided to find by bute force what is the best vectorizer, sampling strategy and model for the given dataset. Additionally, we also tested different representations of the sequences in an attempt of removing any noise we were not aware of. These kinds of knowledge limitations impacted on the results we presented since we are treating the dataset as a black box in certain aspects.

For the experiments, we are trateing the sequences of events as if they were words in a text. Therefore, we treated this problem as text classification and defined the pipeline as such. Therefore, we will test the **TFIDF Vectorier, Count Vectorizer and Word2Vec** to understand whichone can infere the best embedding space for the provided event sequences.

Since we are dealing with a very small data sample (around 1k observations) we wanted to train our models in oversampled data using different sampling methods and find our which can improve the models performance.

---

#### **`__init__(self, X, y)`**

This class is initialized with the training data and the target column. Whithin this method it:
  - Splits `X` (features) and `y` (labels) into training and test sets using **stratified sampling** to ensure class balance.
  - Encodes categorical labels (`y`) into numerical values using `LabelEncoder`.

We also initialize the following attributes to be accessible in all parts of the pipeline:
  - **`self.X_train`, `self.X_test`, `self.y_train`, `self.y_test`:** Prepared splits for training and testing.
  - **`self.trai`ed_models` and `self.trained_vectorizers`:** Empty dictionaries to store trained models and vectorizers.
  - **`self.results`:** Stores evaluation results for comparison.
  - **`sampling_strategies`:** A dictionary of techniques to handle class imbalance, we are using `SMOTE`, `Borderline-SMOTE`, `ADASYN`, `RandomOversampler`, `SMOTE-ENN`, `SMOTE-Tomek`.
  - **`vectorizers`:** A dictionary of methods for text representation, we are using `TFIDF`, `Count`,`Word2Vec`
  - **`classifiers`:** A set of preconfigured machine learning models, such as `LogisticRegression`,`DecisionTree`,`RandomForest`,`ExtraTreesClassifier`,`GradientBoostingClassifier`,`AdaBoostClassifier`,`GaussianNB`,`KNN`,`SVM`,`XGBoost`.

---

#### **`duplicate_minor_classes(self, X, y, min_instances=5)`**

Balances the dataset by duplicating underrepresented classes until they meet the minimum instance threshold.

1. Identifies **minor classes** with fewer samples than `min_instances`.
2. Duplicates rows corresponding to these classes.
3. Merges the duplicated data back with the original dataset.

---

#### **`test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name)`**

Using cross-validation it evaluates a single combination of **Model**, **Vectorizer** and **Sampler**. It returns a list of evaluation metrics (mean and standard deviation) for the specified combination.

1. Splits `X` and `y` into 5 folds using **StratifiedKFold**.
2. For each fold:
   - Transforms the data with the given vectorizer.
   - Balances the data using the sampler.
   - Trains the model on the resampled training data.
   - Predicts on the validation data.
3. Calculates metrics:
   - Accuracy
   - Recall
   - Precision
   - F1 Score


---

#### **`training(self)`**

Runs experiments for all combinations of **Model**, **Vectorizer** and **Sampler**.

1. Balances the training data using `duplicate_minor_classes`.
2. Constructs a task list of all vectorizer-sampler-model combinations.
3. Uses parallel processing to execute each task:
   - For each combination, calls `test()` to evaluate.
4. Aggregates results into a DataFrame for easy comparison.

---

#### **`_process_task(self, task, progress_bar)`**

Handles a single task from the `training()` pipeline to run it in parallel. Returns evaluation metrics for the task.

1. Unpacks the task (vectorizer, sampler, model).
2. Executes `test()` for the task.
3. Updates the progress bar.

---

#### **`test_ensemble(self)`**

Evaluates the performance of an ensemble model across multiple folds. Returns evaluation metrics for the ensemble model.

1. Splits the data using `StratifiedKFold`.
2. For each fold:
   - Trains all models, vectorizers, and samplers on the training data.
   - Constructs an ensemble model using `EnsembleModel`.
   - Tests the ensemble on the validation set.
3. Calculates metrics for the ensemble’s performance.

---

#### **`_process_fold(self, X_train, X_test, y_train, y_test)`**

Handles training and evaluation for a single fold in `test_ensemble` to be executed in parallel along all other folds. Returns evaluation metrics for the ensemble model for that fold.

1. Trains all vectorizers and models on the training data.
2. Stores trained models and vectorizers for the fold.
3. Fits an ensemble model on the predictions.
4. Evaluates the ensemble on the validation set.



## `Word2VecVectorizer`
This class converts sentences into numerical vectors using Word2Vec, which captures the semantic meaning of words. This class was defined to provide an interface similar to Scikit-Learn interface and avoid syntaxis issues later on in the `Experiment` class.

 - `__init__`: Lets you configure how the Word2Vec model works:
   - `size` is the number of dimensions for word vectors (e.g., 100-dimensional space).
   - `window` is the maximum distance between the current and predicted words.
   - `min_count` is the minimum number of times a word must appear to be included in the model.
   - `workers` determines how many CPU threads to use for training (faster with more workers).
 - `fit`: Takes a list of sentences, splits them into words, and trains a Word2Vec model to learn relationships between words.
 - `transform`: Takes sentences and converts them into vectors. It does this by averaging the vectors of all words in a sentence. If a word isn't in the Word2Vec model, it is ignored, and zeros are used instead.
 - `fit_transform`: A convenience method that combines the `fit` (training) and `transform` (vectorizing) steps.

---

In [2]:
class Word2VecVectorizer(TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.w2v_model = None

    def fit(self, X, y=None):
        sentences = [sentence.split() for sentence in X]
        self.w2v_model = Word2Vec(sentences, vector_size=self.size, window=self.window,
                                  min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X, y=None):
        transformed_data = np.array([
            np.mean([self.w2v_model.wv[word] for word in sentence.split() if word in self.w2v_model.wv]
                    or [np.zeros(self.
                                 size)], axis=0)
            for sentence in X
        ])
        return csr_matrix(transformed_data)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [12]:
class NoSampler(TransformerMixin):
    def __init__(self):
      pass

    def fit(self, X, y=None):
        return X, y

    def transform(self, X, y=None):
        return X, y

    def fit_resample(self, X, y=None):
        return X, y

In [13]:
class Experiment:
    def __init__(self, X, y):
        self.X = X
        self.le = LabelEncoder()
        self.y = self.le.fit_transform(y)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
        self.y_train = self.le.transform(self.y_train)
        self.y_test = self.le.transform(self.y_test)
        self.trained_models = {}
        self.trained_vectorizers = {}
        self.trainer_samplers = {}
        self.results = []
        self.sampling_strategies = {
            "SMOTE": SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            "Borderline-SMOTE": BorderlineSMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            "ADASYN": ADASYN(sampling_strategy='auto', random_state=1, n_neighbors=3),
            "RandomOversampler": RandomOverSampler(sampling_strategy='auto', random_state=1),
            "SMOTE-ENN": SMOTEENN(sampling_strategy='auto', random_state=1),
            "SMOTE-Tomek": SMOTETomek(sampling_strategy='auto', random_state=1),
            "NoSamp": NoSampler()
        }

        self.vectorizers = {
            "TFIDF": TfidfVectorizer(),
            "Count": CountVectorizer(),
            "Word2Vec": Word2VecVectorizer(size=100, window=5, min_count=1)
        }
        self.classifiers = {
            'LogisticRegression': LogisticRegression(),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GaussianNB': GaussianNB(),
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(probability=True),
            'XGBoost': XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False, n_jobs = 2),
        }


    def duplicate_minor_classes(self, X, y, min_instances=10):
        X = X.reset_index(drop=True)
        y  = y.reset_index(drop=True)
        class_counts = y.value_counts()

        minor_classes = class_counts[class_counts <= min_instances].index

        minor_class_rows = X.loc[y.isin(minor_classes)]
        minor_class_labels = y.loc[y.isin(minor_classes)]

        duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
        duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

        X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
        y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

        X_balanced.reset_index(drop=True, inplace=True)
        y_balanced.reset_index(drop=True, inplace=True)

        return X_balanced, y_balanced


    def test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        accuracies, recalls, precisions, f1s = [], [], [], []

        for train_index, test_index in skf.split(self.X, self.y):
            X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
            y_train, y_test = self.y[train_index], self.y[test_index]

            X_train, y_train = self.duplicate_minor_classes(X_train, pd.Series(y_train))
            print(y_train.value_counts())

            X_train = vectorizer.fit_transform(X_train).toarray()
            X_test = vectorizer.transform(X_test).toarray()
            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
            model.fit(X_resampled, y_resampled)
            y_pred = model.predict(X_test)
            accuracies.append(accuracy_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred, average="weighted"))
            precisions.append(precision_score(y_test, y_pred, average="weighted"))
            f1s.append(f1_score(y_test, y_pred, average="weighted"))

        return [
            model_name,
            vectorizer_name,
            sampler_name,
            np.mean(accuracies),
            np.std(accuracies),
            np.mean(recalls),
            np.std(recalls),
            np.mean(precisions),
            np.std(precisions),
            np.mean(f1s),
            np.std(f1s),
        ]

    def training(self):
        results = []
        task_list = []

        for vect_name, vectorizer in self.vectorizers.items():
            for samp_name, sampler in self.sampling_strategies.items():
                for clf_name, model in self.classifiers.items():
                    task_list.append(
                        (clf_name, vect_name, samp_name, deepcopy(model), deepcopy(vectorizer), deepcopy(sampler))
                    )

        progress_bar = tqdm(total=len(task_list))
        with parallel_backend("threading"):
            parallel_results = Parallel(n_jobs=-1)(
                delayed(self._process_task)(task, progress_bar) for task in task_list
            )

        results.extend(parallel_results)
        progress_bar.close()

        # Save results and models
        self.results = pd.DataFrame(
            results,
            columns=[
                "Model",
                "Vectorizer",
                "Sampler",
                "Accuracy Mean",
                "Accuracy Std",
                "Recall Mean",
                "Recall Std",
                "Precision Mean",
                "Precision Std",
                "F1 Mean",
                "F1 Std",
            ],
        )
        return self.results

    def _process_task(self, task, progress_bar):
        clf_name, vect_name, samp_name, model, vectorizer, sampler = task
        result = self.test(
            model=model,
            model_name=clf_name,
            vectorizer=vectorizer,
            vectorizer_name=vect_name,
            sampler=sampler,
            sampler_name=samp_name,
        )
        progress_bar.update(1)
        return result

    def test_ensemble(self):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        tasks = []

        for train_index, test_index in skf.split(self.X, self.y):
            X_train, X_test = self.X[train_index], self.X[test_index]
            y_train, y_test = self.y[train_index], self.y[test_index]
            X_train, y_train = self.duplicate_minor_classes(X_train, pd.Series(y_train))
            tasks.append((X_train, X_test, y_train, y_test))

        total_tasks = len(tasks)
        progress_bar = tqdm(total=total_tasks, desc="Processing folds")
        lock = threading.Lock()  # Lock to manage updates safely across threads

        def process_with_progress(*args):
            result = self._process_fold(*args)
            with lock:
                progress_bar.update(1)
            return result

        with parallel_backend("threading"):
            results = Parallel(n_jobs=-1)(
                delayed(process_with_progress)(X_train, X_test, y_train, y_test)
                for X_train, X_test, y_train, y_test in tasks
            )

        progress_bar.close()

        accuracies, recalls, precisions, f1s = zip(*results)

        return [
            "Ensemble", "Multiple", "Multiple",
            np.mean(accuracies), np.std(accuracies),
            np.mean(recalls), np.std(recalls),
            np.mean(precisions), np.std(precisions),
            np.mean(f1s), np.std(f1s)
        ]

    def _process_fold(self, X_train, X_test, y_train, y_test):
        fold_trained_models = {}
        fold_trained_vectorizers = {}

        for vectorizer_name, vectorizer_i in self.vectorizers.items():
            vectorizer = deepcopy(vectorizer_i)
            X_train_vect = vectorizer.fit_transform(X_train).toarray()
            X_test_vect = vectorizer.transform(X_test).toarray()
            fold_trained_vectorizers[vectorizer_name] = vectorizer

            for sampler_name, sampler_i in self.sampling_strategies.items():
                sampler = deepcopy(sampler_i)
                X_resampled, y_resampled = sampler.fit_resample(X_train_vect, y_train)

                for model_name, model in self.classifiers.items():
                    trained_model = deepcopy(model)
                    trained_model.fit(X_resampled, y_resampled)
                    fold_trained_models[(vectorizer_name, sampler_name, model_name)] = trained_model

        ensemble = EnsembleModel(fold_trained_models, fold_trained_vectorizers)
        ensemble.fit(X_train, y_train)

        y_pred = ensemble.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average="weighted")
        precision = precision_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        return accuracy, recall, precision, f1



#`EnsembleModel`

This class combines predictions from multiple models to make a more accurate "group decision."
 - `__init__`: Takes two things:
   - A dictionary of trained models.
   - A dictionary of vectorizers (tools that convert text into numbers).
   It stores these for use during prediction and training.
 - `_generate_prediction_matrix`: This creates a matrix where:
   - Each row corresponds to an input sample.
   - Each column is a model's prediction probabilities for each class.
 - `fit`: Uses an optimization algorithm (`differential_evolution`) to find the best set of weights for combining the model predictions. The goal is to maximize prediction performance (e.g., F1-score).
 - `predict`: Uses the optimized weights to combine predictions from all models and makes the final decision by selecting the class with the highest combined probability.

In [5]:
class EnsembleModel:
    def __init__(self, trained_models, trained_vectorizers):

        self.trained_models = trained_models
        self.trained_vectorizers = trained_vectorizers
        self.optimized_weights = None

    def _generate_prediction_matrix(self, X):
        predictions = {}
        for (vect_name, samp_name, clf_name), model in self.trained_models.items():
            vectorizer = deepcopy(self.trained_vectorizers[vect_name])
            X_vect = vectorizer.transform(X).toarray()

            predictions[(vect_name, samp_name, clf_name)] = model.predict_proba(X_vect)

        return np.stack(list(predictions.values()), axis=2)

    def fit(self, X_train, y_train):
        pred_matrix = self._generate_prediction_matrix(X_train)

        def fitness(weights):
            weighted_pred = np.tensordot(pred_matrix, weights, axes=([2], [0]))
            final_pred = np.argmax(weighted_pred, axis=1)
            return -f1_score(y_train, final_pred, average='weighted')

        num_models = pred_matrix.shape[2]
        bounds = [(0, 1)] * num_models
        result = differential_evolution(fitness, bounds)

        self.optimized_weights = result.x

    def predict(self, X):
        if self.optimized_weights is None:
            raise ValueError("The ensemble model must be trained using `train_ensemble` before prediction.")

        pred_matrix = self._generate_prediction_matrix(X)
        weighted_pred = np.tensordot(pred_matrix, self.optimized_weights, axes=([2], [0]))
        ensemble_pred = np.argmax(weighted_pred, axis=1)
        return ensemble_pred


# `Representations`

This class creates different ways to represent sequences of events for machine learning analysis.
 - `representation_a`: Focuses on filtering events:
   - Counts the frequency of each event in all sequences.
   - Keeps only the events that occur less than 85% of the time, filtering out common noise.
   - Outputs a "cleaned" sequence of events and the labels (e.g., incident types).
 - `representation_b`: Splits event sequences into two parts:
   - Events *before* an incident (labeled with the incident type).
   - Events *after* an incident (labeled as "unknown").
 - `representation_c`: Breaks sequences into overlapping chunks:
   - For example, if you have a sequence of 50 events, it divides them into smaller parts (e.g., chunks of 30 events with 15 overlapping).
   - Each chunk is labeled with the incident type.

---

In [6]:
class Representations:
  def __init__(self, df):
    self.df = df

  def representation_a(self, df):
            events_types_dict = {}
            for events_sequence in df['events_sequence']:
                row_list = ast.literal_eval(events_sequence)
                unique_events = set(row_list)
                for event in unique_events:
                    if not events_types_dict.get(event):
                        events_types_dict[event] = 0
                    events_types_dict[event] += 1

            sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
            sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
            sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / df.shape[0] * 100
            sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)

            events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage <= 85].event_type)))

            df['clean_events_sequence'] = (
                df['events_sequence']
                .apply(ast.literal_eval)
                .apply(lambda x: [i for i in x if i in events_low_frequency])
                .astype(str)
                .replace(r'[\[\],]', '', regex=True)
            )

            self.y = df['incident_type'].copy()
            self.X = df['clean_events_sequence'].copy()
            return self.X, self.y

  def representation_b(self, df):

    before_incident = []
    after_incident = []

    for _, row in df.iterrows():
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        before_events = " ".join([str(event) for event, time in zip(events, seconds) if time <= 0])
        if before_events:
            before_incident.append({
                "events_sequence": before_events,
                "class": incident_type
            })

        after_events = " ".join([str(event) for event, time in zip(events, seconds) if time > 0])
        if after_events:
            after_incident.append({
                "events_sequence": after_events,
                "class": 100
            })

    before_df = pd.DataFrame(before_incident)
    after_df = pd.DataFrame(after_incident)

    return before_df, after_df


  def representation_c(self, df, sequence_length=30):

    overlapping_sequences = []
    step = sequence_length // 2

    for _, row in df.iterrows():
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        for i in range(0, len(events) - sequence_length + 1, step):
            sequence = [str(i) for i in events[i:i + sequence_length]]
            seconds_slice = seconds[i:i + sequence_length]
            sequence_class = incident_type

            overlapping_sequences.append({"sequence": " ".join(sequence), "class": sequence_class})

    sequences_df = pd.DataFrame(overlapping_sequences)
    return sequences_df.sequence, sequences_df['class']


# Experiments

In [7]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';', index_col=0)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
357,4450563,"[1033, 1033, 1033, 1033, 1033, 1033, 1033, 103...","[4004, 4032, 4028, 2852, 4026, 4110, 2854, 409...","[-7370, -7370, -7369, -7368, -7368, -7368, -73...",50.146926,5.977864,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",11
244,4444499,"[595, 595, 595, 595, 595, 595, 595, 595, 595, ...","[4396, 690, 2564, 2534, 2492, 2494, 4102, 4106...","[-12869, -12869, -12867, -12746, -12569, -1226...",32.479336,3.387316,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, False, False, False, False,...",14


## 1. Raw Dataset
We want to test the performance of the pre-defined pipeline as a baseline for future experiments. That is, the data won't have any further transformation other than those defined in the `Experiment` class.

In [14]:
X, y = df['events_sequence'], df.incident_type
exp = Experiment(X, y)
results = exp.training()


  0%|          | 0/210 [00:00<?, ?it/s]

7     254
11    140
8     120
0      95
5      93
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     254
11    140
8     120
0      95
5      93
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      93
2      62
10     24
6      21
1      12
9      12
3      12
4      12
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      94
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      93
2      62
10     24
6      21
1      12
9      12
3      12
4      12
Name: count, dtype: int64
7     254
11    140
8     119
0      95
5      94
2      63
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      94
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name


  0%|          | 1/210 [00:08<30:09,  8.66s/it]

7     254
11    140
8     120
0      95
5      93
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     254
11    140
8     119
0      95
5      94
2      63
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      93
2      62
10     24
6      21
1      12
9      12
3      12
4      12
Name: count, dtype: int64
7     254
11    140
8     119
0      96
5      94
2      63
10     24
6      20
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      94
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64



  1%|          | 2/210 [00:16<29:16,  8.45s/it]

...


 99%|█████████▉| 208/210 [3:55:57<01:29, 44.55s/it]

7     254
11    140
8     120
0      95
5      93
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     254
11    140
8     119
0      95
5      94
2      63
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     254
11    140
8     119
0      96
5      94
2      63
10     24
6      20
3      15
1      12
9       9
4       9
Name: count, dtype: int64



100%|█████████▉| 209/210 [3:56:11<00:35, 35.44s/it]

7     255
11    140
8     119
0      95
5      93
2      62
10     24
6      21
1      12
9      12
3      12
4      12
Name: count, dtype: int64
7     255
11    140
8     119
0      95
5      94
2      62
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     254
11    140
8     119
0      95
5      94
2      63
10     24
6      21
3      15
1      12
9       9
4       9
Name: count, dtype: int64
7     254
11    140
8     119
0      96
5      94
2      63
10     24
6      20
3      15
1      12
9       9
4       9
Name: count, dtype: int64



100%|██████████| 210/210 [3:56:48<00:00, 67.66s/it]


As you can see, after trainign 180 models with different parameters, we can see that the `GradientBoostingClassifier` had an outstanding performance in comparison to the other models. We'll take the results of a 73.46% mean F1-score and 1.7% std in the K-Fold validation as our baseline for future comparisons.

In [15]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
104,GradientBoostingClassifier,Count,RandomOversampler,0.694328,0.021527,0.694328,0.021527,0.708198,0.019593,0.690832,0.020778
94,GradientBoostingClassifier,Count,ADASYN,0.698312,0.011431,0.698312,0.011431,0.694056,0.011840,0.689301,0.011707
84,GradientBoostingClassifier,Count,Borderline-SMOTE,0.689411,0.016173,0.689411,0.016173,0.689645,0.020071,0.682727,0.015752
74,GradientBoostingClassifier,Count,SMOTE,0.687412,0.026736,0.687412,0.026736,0.690061,0.025702,0.681560,0.026748
124,GradientBoostingClassifier,Count,SMOTE-Tomek,0.680500,0.018637,0.680500,0.018637,0.682200,0.017600,0.675170,0.017216
34,GradientBoostingClassifier,TFIDF,RandomOversampler,0.679462,0.035520,0.679462,0.035520,0.686762,0.028207,0.675155,0.033172
54,GradientBoostingClassifier,TFIDF,SMOTE-Tomek,0.677520,0.033279,0.677520,0.033279,0.701064,0.023753,0.674030,0.030149
64,GradientBoostingClassifier,TFIDF,NoSamp,0.681471,0.030430,0.681471,0.030430,0.697387,0.031089,0.673383,0.029483
4,GradientBoostingClassifier,TFIDF,SMOTE,0.669585,0.026967,0.669585,0.026967,0.693897,0.022211,0.667261,0.024788
134,GradientBoostingClassifier,Count,NoSamp,0.672580,0.016485,0.672580,0.016485,0.682098,0.016819,0.664670,0.014907


In [ ]:
ensemble_results = pd.DataFrame(exp.test_ensemble()).T
ensemble_results.columns = ["Model","Vectorizer","Sampler","Accuracy Mean","Accuracy Std","Recall Mean","Recall Std","Precision Mean","Precision Std","F1 Mean","F1 Std"]


Processing folds:   0%|          | 0/5 [00:00<?, ?it/s]

None the less, using the 180 models to build an ensemble model did not improve the results of the F1 score, since it only achieved a 69.53% mean F1-score with 2.11% of std.

In [ ]:
ensemble_results

## 2. Representation A: Keep only the events that occur less than 85% of the time
We took the paper provided by SCNB as inspiration to filter out some inherent noise that could affect the results of the previous experiment.

In [ ]:
X, y = Representations(df).representation_a(df)
exp = Experiment(X, y)
results = exp.training()

From the following table we can see that filtering out the noise negatively impacted the model, since this time we achieved a 3 percentag points lower mean F1-score than our base line, i.e. 70.65% with an almost 3x greater standar deviation.

In [ ]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

In [ ]:
ensemble_results = pd.DataFrame(exp.test_ensemble()).T
ensemble_results.columns = ["Model","Vectorizer","Sampler","Accuracy Mean","Accuracy Std","Recall Mean","Recall Std","Precision Mean","Precision Std","F1 Mean","F1 Std"]

In this case, we built an ensemble model to test if we could achieve a better mean F1-score. Unlike the 1st experiment, we achieced a mea F1-score 2 percentage points greater than the ensemble of the 1st experiment. Nonetheless, it was not enought to outperform the baseline model and also it had a slightly greater standard deviation.

In [ ]:
ensemble_results

## Representation B: Split events into those before and after the incident

In [ ]:
df_before, df_after = Representations(df).representation_b(df)
X, y = pd.concat([df_before.events_sequence, df_after.events_sequence]), pd.concat([df_before['class'], df_after['class']])
exp = Experiment(X, y)
results = exp.training()

In [ ]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

In [ ]:
ensemble_results = pd.DataFrame(exp.test_ensemble()).T
ensemble_results.columns = ["Model","Vectorizer","Sampler","Accuracy Mean","Accuracy Std","Recall Mean","Recall Std","Precision Mean","Precision Std","F1 Mean","F1 Std"]

In [ ]:
ensemble_results

## Representation C: Generates fixed length overlappng and non-overlaping sequences out of the provided sequences

In [ ]:
X, y = Representations(df).representation_c(df, sequence_length=100)
exp = Experiment(X, y)
results = exp.training()

In [ ]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

In [ ]:
ensemble_results = pd.DataFrame(exp.test_ensemble()).T
ensemble_results.columns = ["Model","Vectorizer","Sampler","Accuracy Mean","Accuracy Std","Recall Mean","Recall Std","Precision Mean","Precision Std","F1 Mean","F1 Std"]

In [ ]:
ensemble_results

# Conclusion